In [8]:
import sys
import os
import math
import numpy as np
import torch
from torchvision.models import vit_b_16
from external.wdpruning.vit_wdpruning import VisionTransformerWithWDPruning
import torch.nn as nn


sys.path.append(os.path.abspath(os.path.join(os.path.dirname("ide"), '../..')))
#sys.path.append(os.path.abspath(os.path.join(os.path.dirname("deep_feature_reweighting"), '../..')))

In [21]:
device = torch.device('cuda')

ckpt_pruned = torch.load("/home/primmere/ide/external/wdpruning/logs/wb/r90e50val/checkpoint-best.pth", map_location=device,weights_only=False)['model']
ckpt_orig = torch.load("/home/primmere/ide/dfr/logs/vit_waterbirds.pth", map_location=device,weights_only=False)

model_pruned = VisionTransformerWithWDPruning(num_classes=2,
                                       patch_size=16, embed_dim=768,
                                       depth=12, num_heads=12, mlp_ratio=4,
                                       head_pruning=True, fc_pruning=True)
model_pruned._make_structural_pruning()
model_pruned.to(device)

model_orig = vit_b_16(weights=None)
model_orig.heads.head = nn.Linear(model_orig.heads.head.in_features, 2)
model_orig.load_state_dict(ckpt_orig)
model_orig.to(device)

using stride: 16, and patch number is num_y14 * num_x14
using drop_out rate is : 0.0
using attn_drop_out rate is : 0.0
using drop_path rate is : 0.1


VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [22]:
for (name_a, p_a), (name_b, p_b) in zip(model_orig.named_parameters(),
                                        model_pruned.named_parameters()):
    # --- NEW filter ---------------------------------------------------
    if "blocks" not in name_b:            # only look at blocks tensors
        continue
    if "saliency" in name_b or "threshold" in name_b:
        raise ValueError(  f"shouldnt exist: {name_b}" )                       # skip the unwanted ones
    # -----------------------------------------------------------------

    if not torch.allclose(p_a, p_b, rtol=1e-5, atol=1e-8):
        print(f"{name_a}, {name_b} differs!")

encoder.layers.encoder_layer_0.ln_1.weight, blocks.0.norm1.weight differs!
encoder.layers.encoder_layer_0.ln_1.bias, blocks.0.norm1.bias differs!
encoder.layers.encoder_layer_0.self_attention.in_proj_weight, blocks.0.attn.qkv.weight differs!
encoder.layers.encoder_layer_0.self_attention.in_proj_bias, blocks.0.attn.qkv.bias differs!
encoder.layers.encoder_layer_0.self_attention.out_proj.weight, blocks.0.attn.proj.weight differs!
encoder.layers.encoder_layer_0.self_attention.out_proj.bias, blocks.0.attn.proj.bias differs!
encoder.layers.encoder_layer_0.ln_2.weight, blocks.0.norm2.weight differs!
encoder.layers.encoder_layer_0.ln_2.bias, blocks.0.norm2.bias differs!
encoder.layers.encoder_layer_0.mlp.0.weight, blocks.0.mlp.fc1.weight differs!
encoder.layers.encoder_layer_0.mlp.0.bias, blocks.0.mlp.fc1.bias differs!
encoder.layers.encoder_layer_0.mlp.3.weight, blocks.0.mlp.fc2.weight differs!
encoder.layers.encoder_layer_0.mlp.3.bias, blocks.0.mlp.fc2.bias differs!
encoder.layers.encoder_l

In [23]:
for (name_a, p_a), (name_b, p_b) in zip(model_orig.named_parameters(),
                                        model_pruned.named_parameters()):
    if "blocks" not in name_b:         
        continue
    if "saliency" in name_b or "threshold" in name_b:
        continue 
    print("xxx")
    print(name_a)
    print(name_b)
    print("xxx")


xxx
encoder.layers.encoder_layer_0.ln_1.weight
blocks.0.norm1.weight
xxx
xxx
encoder.layers.encoder_layer_0.ln_1.bias
blocks.0.norm1.bias
xxx
xxx
encoder.layers.encoder_layer_0.self_attention.in_proj_weight
blocks.0.attn.qkv.weight
xxx
xxx
encoder.layers.encoder_layer_0.self_attention.in_proj_bias
blocks.0.attn.qkv.bias
xxx
xxx
encoder.layers.encoder_layer_0.self_attention.out_proj.weight
blocks.0.attn.proj.weight
xxx
xxx
encoder.layers.encoder_layer_0.self_attention.out_proj.bias
blocks.0.attn.proj.bias
xxx
xxx
encoder.layers.encoder_layer_0.ln_2.weight
blocks.0.norm2.weight
xxx
xxx
encoder.layers.encoder_layer_0.ln_2.bias
blocks.0.norm2.bias
xxx
xxx
encoder.layers.encoder_layer_0.mlp.0.weight
blocks.0.mlp.fc1.weight
xxx
xxx
encoder.layers.encoder_layer_0.mlp.0.bias
blocks.0.mlp.fc1.bias
xxx
xxx
encoder.layers.encoder_layer_0.mlp.3.weight
blocks.0.mlp.fc2.weight
xxx
xxx
encoder.layers.encoder_layer_0.mlp.3.bias
blocks.0.mlp.fc2.bias
xxx
xxx
encoder.layers.encoder_layer_1.ln_1.weight
b